In [3]:
import pickle

In [4]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.insert(0, 'src')

%aimport prolog

### People list

In [152]:
# #### Extract people... ?
# 
# from interval import interval
# 
# text = " ".join(s["source"] for s in dataset)
# 
# # Find Cap names
# cap_names = list(re.finditer("(?:[a-z][,:; ]+)([A-Z][a-z]*(?:\s+[A-Z][a-z]*)*)", text))
# # Find Mr., Mrs., Sir. names
# title_names = list(re.finditer("([Mm](?:rs?\.?|iss\.?|[Ss]ir)\s+[A-Z][a-z]*(?:\s+[A-Z][a-z]*)*)", text))
# 
# characters_spans = interval(*[n.span(1) for n in cap_names] + [n.span(0) for n in title_names])
# 
# # {text[int(s[0]):int(s[1])]: ((int(s[0]), int(s[1]), text[int(s[0])-10:int(s[1])+20])) for s in characters_spans}

For now we just hard code the known list of protagonists (corrected from the given list in the article)

`people_list = [...]`

In [333]:
people_list = pickle.load(open("corpus/people.pkl", "rb"))

In [154]:
people_code_to_name = {p["code"]: p["main"] for p in people_list}

In [155]:
%aimport preprocessing

#### Book utterances splitting and annotations matching

In [171]:
utterances = preprocessing.build_dataset(
    text_file='corpus/PRIDPREJ_NONEWLINE_Organize_v2.txt',
    people=people_list
)
dataset = preprocessing.match_with_annoted_file(
    path='corpus/REAL_ALL_CONTENTS_PP.txt',
    utterances=utterances,
    people=people_list)

Almost match :
My dear Elizabeth_Bennet, I have the highest opinion in the world of your excellent judgment in all matters within the scope of your understanding, but permit me to say that there must be a wide difference between the established forms of ceremony amongst the laity, and those which regulate the clergy; for give me leave to observe that I consider the clerical office as equal in point of dignity with the highest rank in the kingdom -- provided that a proper humility of behaviour is at the same time maintained. You must therefore allow me to follow the dictates of my conscience on this occasion, which leads me to perform what I look on as a point of duty. Pardon me for neglecting to profit by your advice, which on every other subject shall be my constant guide, though in the case before us I consider myself more fitted by education and habitual study to decide on what is right than a young lady like yourself. [X] apology, [X] Hunsford, [X] Lady_Catherine.
My dear Elizabeth

In [173]:
dataset[0]

{'begin': 0,
 'discussion_index': 0,
 'end': 109,
 'only_utterance_article': 'My dear Mr_Bennet, [X] have you heard that Netherfield Park is let at last?',
 'only_utterance_us': 'My dear Mr_Bennet, [X] have you heard that Netherfield Park is let at last?',
 'parts': [{'text': 'My dear Mr_Bennet,', 'utterance': True},
  {'text': ' said his lady to him one day, ', 'utterance': False},
  {'text': 'have you heard that Netherfield Park is let at last?',
   'utterance': True}],
 'source': "``My dear Mr_Bennet,'' said his lady to him one day, ``have you heard that Netherfield Park is let at last?''",
 'target': 'Mrs_Bennet'}

### Grammar parsing

Let's detect the gender/role/name features in the narration parts

In [179]:
import nltk
from nltk.parse import stanford

jar = '../stanford-parser-full-2017-06-09/stanford-parser.jar'
model = '../stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar'

dep_parser = stanford.StanfordDependencyParser(model, jar, model_path='edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz', encoding='utf8')

Add standford parser annotations like name or gender


In [180]:
import tqdm

In [181]:
#######################################################
# Stanford parser rules: triples -> name, gender, etc #
#######################################################
def extract_features(triple):
    (word1, tag1), dep, (word2, tag2) = triple
    if (tag1.startswith('VBD') or word1 in ['said', 'added', 'cried', 'asked', 'replied', 'returned', 'continued', 'observed']) and (tag2.startswith('NN') or tag2.startswith('PRP')) and not dep.startswith('nmod'):
        if tag2.startswith('NN'):
            if tag2.startswith('NNP'):
                speaker_name = word2
            else:
                speaker_role = word2
        if word2 in ['he', 'man', 'boy', 'lad']:
            speaker_gender = 'M'
        if word2 in ['she', 'lady', 'girl']:
            speaker_gender = 'F'
    return (speaker_name, speaker_role, speaker_gender)


for sample in tqdm.tqdm(dataset):
    for part in sample["parts"]:
        if not part["utterance"]:
            speaker_name = None
            speaker_role = None
            speaker_gender = None
            tokens = nltk.word_tokenize(re.sub("[^_\w]", "", part["text"][:200]))
            tagged = nltk.pos_tag(tokens)
            try:
                dependencies = sum([[list(parse.triples()) for parse in dep_graphs] for dep_graphs in dep_parser.tagged_parse_sents([tagged])],[])
                for (term1,dep,term2) in dependencies[0]:
                    #print(term1, dep, term2)
                    speaker_name, speaker_role, speaker_gender = extract_features((term1,dep,term2) )
                    # try reverse order
                    speaker_name, speaker_role, speaker_gender =  extract_features((term2,dep,term1) )
            except Exception as e:
                    print(e)
                    print(part)
            part["speaker_name"] = speaker_name
            part["speaker_role"] = speaker_role
            part["speaker_gender"] = speaker_gender
            #print(part)

100%|██████████| 1302/1302 [41:05<00:00,  1.89s/it]


In [182]:
pickle.dump(dataset, open("corpus/dataset-parser-features.pkl", "wb"))

In [183]:
true_roles = {
    "husband": "husband",
    "lady": "wife",
    "wife": "wife",
    "aunt": "aunt",
    "uncle": "uncle",
    "father": "father",
    "sister": "sister",
    "mother": "mother",
    "daughter": "daughter",
    "son": "son",
    "cousin": "cousin",
}

In [184]:
import re

### Load Prolog engine

Define facts

`facts = [...]`

In [185]:
facts = """
status(mrs_annesley,female).
status(elizabeth_bennet,female).
status(jane_bennet,female).
status(lydia_bennet,female).
status(kitty_bennet,female).
status(mary_bennet,female).
status(mrs_bennet,female).
status(mr_bennet,male).
status(mr_bingley,male).
status(caroline_bingley,female).
status(charlotte,female).
status(captain_carter,female).
status(mr_collins,male).
status(lady_catherine,female).
status(mr_chamberlayne,female).
status(dawson,female).
status(mr_denny,female).
status(mr_darcy,male).
status(old_mr_darcy,female).
status(lady_anne_darcy,female).
status(georgiana_darcy,female).
status(colonel_fitzwilliam,male).
status(colonel_forster,female).
status(miss_grantley,female).
status(mrs_gardiner,female).
status(mr_gardiner,male).
status(william_goulding,female).
status(haggerston,female).
status(mrs_hill,female).
status(mrs_jenkinson,female).
status(mr_jones,female).
status(miss_mary_king,female).
status(mrs_long,female).
status(lady_lucas,female).
status(maria_lucas,female).
status(mr_hurst,female).
status(louisa_hurst,female).
status(lady_metcalfe,female).
status(mr_morris,female).
status(mrs_nicholls,female).
status(mr_philips,male).
status(miss_pope,female).
status(mr_pratt,male).
status(mrs_reynolds,female).
status(mr_robinson,female).
status(mr_stone,female).
status(miss_watson,female).
status(old_mr_wickham,female).
status(sir_william,male).
status(anne_de_bourgh,female).
status(mr_wickham,male).
status(mrs_philips,female).
status(young_lucas,male).
status(the_butler,male).

status(elizabeth_bennet,female).
related(jane_bennet,elizabeth_bennet,sister).
related(mary_bennet,elizabeth_bennet,sister).
related(lydia_bennet,elizabeth_bennet,sister).
related(kitty_bennet,elizabeth_bennet,sister).
related(mr_bennet,elizabeth_bennet,father).
related(mrs_bennet,elizabeth_bennet,mother).
related(mr_collins,charlotte,husband).
related(mr_collins,mr_bennet,brother).
related(mr_bingley,caroline_bingley,siblings).
"""

In [186]:
if "prolog_engine" in globals() and not prolog_engine.closed:
    prolog_engine.close()
prolog_engine = prolog.Prolog('./family.pl')
prolog_engine.run()

In [187]:
prolog_engine.assert_facts(facts) # input the facts
prolog_engine.query("abolish_all_tables.") # reset the cache

True

Test a query

In [188]:
prolog_engine.query('status(X,female),related(X,_,wife).')

[{'X': 'mrs_bennet'}, {'X': 'charlotte'}]

In [203]:
dl_dataset = []
for utterance in tqdm.tqdm(dataset):
    parts = utterance['parts']
    parts_count = len(parts)
    
    if " and " in utterance['target']:
        print(utterance['target'], len(parts))
    speaker = utterance['target']
    discussion_index = utterance['discussion_index']
    
    utterance_text = re.sub('\[X\]', '', utterance['only_utterance_us'])
    incises = [part for part in parts if not part['utterance']]
    # for i in range(len(parts)):
    #     if not parts[i]['utterance']:
    #         continue
    #     incises = (
    #         [parts[i-1]] if i > 0 and not parts[i-1]['utterance'] else []
    #         + [parts[i+1]] if i<parts_count-1 and not parts[i+1]['utterance'] else [])

    concat_incise = "".join(part["text"] for part in incises)
    role = next((true_roles[part["speaker_role"]] for part in incises if part["speaker_role"] in true_roles), None)
    gender = next((part["speaker_gender"] for part in incises), None)
    name = next((part["speaker_name"] for part in incises), None)

    potential_targets = []
    query = []
    if name is not None:
        potential_targets = [name]
    else:
        if gender is not None:
            query.append("status(X,{})".format("female" if gender is "F" else "male"))
        if role is not None:
            query.append("related(X,_,{})".format(role))
        if len(query)>0:
            query = ",".join(query)+"."
            res = prolog_engine.query(query)
            if isinstance(res, list):
                potential_targets = set([people_code_to_name[r['X']] for r in res])
            else:
                potential_targets = set()

    dl_dataset.append((discussion_index, utterance_text, concat_incise, potential_targets, speaker, role, gender, name))

100%|██████████| 1302/1302 [06:54<00:00,  3.14it/s]


In [330]:
tokens = nltk.word_tokenize(re.sub("[^_\w ]+", "", "her nice mother said to him, "))
tagged = nltk.pos_tag(tokens)
sum([[list(parse.triples()) for parse in dep_graphs] for dep_graphs in dep_parser.tagged_parse_sents([tagged])],[])

[[(('said', 'VBD'), 'nsubj', ('mother', 'NN')),
  (('mother', 'NN'), 'nmod:poss', ('her', 'PRP$')),
  (('mother', 'NN'), 'amod', ('nice', 'JJ')),
  (('said', 'VBD'), 'nmod', ('him', 'PRP')),
  (('him', 'PRP'), 'case', ('to', 'TO'))]]

### Split the dataset in multiple discussions

In [331]:
dataset_discussions = []
current_discussion = []
current_discussion_index = None
for sample in dl_dataset:
    if sample[0] != current_discussion_index and len(current_discussion) > 0:
        dataset_discussions.append(current_discussion)
        current_discussion = []
    current_discussion_index = sample[0]
    current_discussion.append(sample[1:])
if len(current_discussion) > 0:
    dataset_discussions.append(current_discussion)

In [332]:
pickle.dump(dataset_discussions, open("corpus/dataset-prolog.pkl", "wb"))

#### Prediction only based on parsing and ontology results

In [292]:
import numpy as np

In [317]:
correct_answers = 0
people_names = [p['main'] for p in people_list]
for discussion in dataset_discussions:
    for sample in discussion:
        if len(sample[2]) > 0 and np.random.choice(list(sample[2]), 1) == sample[3]:
            correct_answers += 1
print("accuracy: {}".format(correct_answers/len(dl_dataset)))

accuracy: 0.22964669738863286


#### Prediction only based on parsing and ontology results and nearby speakers

In [326]:
correct_answers = 0
people_names = [p['main'] for p in people_list]
for discussion in dataset_discussions:
    discussion_people = set()
    previous_predictions = [] 
    for i, sample in enumerate(discussion):
        current_predictions = set()
        if len(sample[2]) == 1:
            current_predictions = {next(iter(sample[2]))}
            discussion_people |= current_predictions
        elif len(sample[2]) > 1:
            current_predictions = sample[2] & discussion_people
            around_predictions = set()
            if i >= 2:
                 around_predictions |= set(discussion[i-2][2])
            if i < len(discussion)-2:
                 around_predictions |= set(discussion[i+2][2])
            # if i >= 4:
            #      around_predictions |= set(discussion[i-4][2])
            # if i < len(discussion)-4:
            #      around_predictions |= set(discussion[i+4][2])
            
            if len(current_predictions) > 1:
                if len(current_predictions & around_predictions) >= 1:
                    current_predictions = current_predictions & around_predictions
            elif len(current_predictions) == 0:
                current_predictions = around_predictions
        if len(current_predictions) == 1:
            previous_predictions.append(next(iter(current_predictions)))
        else:
            previous_predictions.append(None)
            
        if len(current_predictions) > 0 and np.random.choice(list(current_predictions), 1) == sample[3]:
            correct_answers += 1
        else:
            pass
            
print("accuracy: {}".format(correct_answers/len(dl_dataset)))

accuracy: 0.2780337941628264
